# Polars vs Pandas Comparison

This notebook compares Polars and Pandas using NBA data.

In [1]:
import polars as pl
import pandas as pd
import time
import os

---

## 1. Loading Data

In [2]:
DATA_PATH = "./archive/raw/Games.csv"

In [3]:
# Load with Polars
start = time.perf_counter()
df_polars = pl.read_csv(DATA_PATH)
polars_load_time = time.perf_counter() - start
print(f"Polars load time: {polars_load_time:.4f}s")
print(f"Shape: {df_polars.shape}")

Polars load time: 0.0224s
Shape: (72524, 17)


In [4]:
# Load with Pandas
start = time.perf_counter()
df_pandas = pd.read_csv(DATA_PATH)
pandas_load_time = time.perf_counter() - start
print(f"Pandas load time: {pandas_load_time:.4f}s")
print(f"Shape: {df_pandas.shape}")

Pandas load time: 0.1875s
Shape: (72524, 17)


/tmp/ipykernel_223319/3417527679.py:3: DtypeWarning: Columns (0: gameSubLabel) have mixed types. Specify dtype option on import or set low_memory=False.
  df_pandas = pd.read_csv(DATA_PATH)


In [5]:
print(f"\n📊 Loading: Polars is {pandas_load_time/polars_load_time:.2f}x faster")


📊 Loading: Polars is 8.36x faster


---

## 2. Selecting Columns

In [6]:
# Polars
start = time.perf_counter()
result_polars = df_polars.select(["gameId", "hometeamName", "awayteamName", "homeScore", "awayScore"])
polars_time = time.perf_counter() - start
print(f"Polars: {polars_time:.6f}s")

Polars: 0.002957s


In [7]:
# Pandas
start = time.perf_counter()
result_pandas = df_pandas[["gameId", "hometeamName", "awayteamName", "homeScore", "awayScore"]]
pandas_time = time.perf_counter() - start
print(f"Pandas: {pandas_time:.6f}s")

Pandas: 0.001735s


In [8]:
print(f"\n📊 Select columns: Polars is {pandas_time/polars_time:.2f}x faster")


📊 Select columns: Polars is 0.59x faster


---

## 3. Filtering Rows

In [9]:
# Polars - filter games where home team scored more than 120
start = time.perf_counter()
result_polars = df_polars.filter(pl.col("homeScore") > 120)
polars_time = time.perf_counter() - start
print(f"Polars: {polars_time:.6f}s - {len(result_polars)} rows")

Polars: 0.007663s - 11140 rows


In [10]:
# Pandas - filter games where home team scored more than 120
start = time.perf_counter()
result_pandas = df_pandas[df_pandas["homeScore"] > 120]
pandas_time = time.perf_counter() - start
print(f"Pandas: {pandas_time:.6f}s - {len(result_pandas)} rows")

Pandas: 0.006054s - 11140 rows


In [11]:
print(f"\n📊 Filter rows: Polars is {pandas_time/polars_time:.2f}x faster")


📊 Filter rows: Polars is 0.79x faster


---

## 4. Multiple Filters (Chained)

In [12]:
# Polars - chain multiple filters
start = time.perf_counter()
result_polars = (
    df_polars
    .filter(pl.col("homeScore") > 120)
    .filter(pl.col("awayScore") > 100)
    .filter(pl.col("hometeamName") == "Lakers")
)
polars_time = time.perf_counter() - start
print(f"Polars: {polars_time:.6f}s - {len(result_polars)} rows")

Polars: 0.010913s - 661 rows


In [13]:
# Pandas - multiple filters
start = time.perf_counter()
result_pandas = df_pandas[
    (df_pandas["homeScore"] > 120) & 
    (df_pandas["awayScore"] > 100) & 
    (df_pandas["hometeamName"] == "Lakers")
]
pandas_time = time.perf_counter() - start
print(f"Pandas: {pandas_time:.6f}s - {len(result_pandas)} rows")

Pandas: 0.008934s - 661 rows


In [14]:
print(f"\n📊 Multiple filters: Polars is {pandas_time/polars_time:.2f}x faster")


📊 Multiple filters: Polars is 0.82x faster


---

## 5. Group By + Aggregation

In [15]:
# Polars - average score by home team
start = time.perf_counter()
result_polars = (
    df_polars
    .group_by("hometeamName")
    .agg(
        pl.col("homeScore").mean().alias("avg_home_score"),
        pl.col("homeScore").count().alias("games_count")
    )
    .sort("avg_home_score", descending=True)
)
polars_time = time.perf_counter() - start
print(f"Polars: {polars_time:.6f}s")

Polars: 0.010738s


In [16]:
# Pandas - average score by home team
start = time.perf_counter()
result_pandas = (
    df_pandas
    .groupby("hometeamName")["homeScore"]
    .agg(["mean", "count"])
    .reset_index()
    .rename(columns={"mean": "avg_home_score", "count": "games_count"})
    .sort_values("avg_home_score", ascending=False)
)
pandas_time = time.perf_counter() - start
print(f"Pandas: {pandas_time:.6f}s")

Pandas: 0.012501s


In [17]:
print(f"\n📊 Group by + aggregation: Polars is {pandas_time/polars_time:.2f}x faster")


📊 Group by + aggregation: Polars is 1.16x faster


---

## 6. Adding/Deriving Columns

In [18]:
# Polars - add computed column
start = time.perf_counter()
result_polars = df_polars.with_columns(
    (pl.col("homeScore") + pl.col("awayScore")).alias("total_score"),
    (pl.col("homeScore") - pl.col("awayScore")).alias("score_diff")
)
polars_time = time.perf_counter() - start
print(f"Polars: {polars_time:.6f}s")

Polars: 0.009702s


In [19]:
# Pandas - add computed column
start = time.perf_counter()
result_pandas = df_pandas.copy()
result_pandas["total_score"] = result_pandas["homeScore"] + result_pandas["awayScore"]
result_pandas["score_diff"] = result_pandas["homeScore"] - result_pandas["awayScore"]
pandas_time = time.perf_counter() - start
print(f"Pandas: {pandas_time:.6f}s")

Pandas: 0.010830s


In [20]:
print(f"\n📊 Add columns: Polars is {pandas_time/polars_time:.2f}x faster")


📊 Add columns: Polars is 1.12x faster


---

## 7. Handling Nulls

In [21]:
# Polars - fill nulls and drop nulls
start = time.perf_counter()
result_polars = df_polars.fill_null("Unknown").drop_nulls()
polars_time = time.perf_counter() - start
print(f"Polars: {polars_time:.6f}s")

Polars: 0.006279s


In [22]:
# Pandas - fill nulls and drop nulls
start = time.perf_counter()
result_pandas = df_pandas.fillna("Unknown").dropna()
pandas_time = time.perf_counter() - start
print(f"Pandas: {pandas_time:.6f}s")

Pandas: 0.083017s


In [23]:
print(f"\n📊 Handle nulls: Polars is {pandas_time/polars_time:.2f}x faster")


📊 Handle nulls: Polars is 13.22x faster


---

## 8. Complex Query: Lazy Evaluation (Polars)

In [24]:
# Polars Lazy - build query without executing
query = (
    pl.scan_csv(DATA_PATH)
    .filter(pl.col("homeScore") > 100)
    .group_by("hometeamName")
    .agg(
        pl.col("homeScore").mean().alias("avg_score"),
        pl.col("gameId").count().alias("games")
    )
    .sort("avg_score", descending=True)
    .limit(10)
)
print("Polars Lazy query plan:")
print(query.explain())

Polars Lazy query plan:
SORT BY [col("avg_score")]
  AGGREGATE[maintain_order: false]
    [col("homeScore").mean().alias("avg_score"), col("gameId").count().alias("games")] BY [col("hometeamName")]
    FROM
    Csv SCAN [./archive/raw/Games.csv] [id: 140735594249456]
    PROJECT 3/17 COLUMNS
    SELECTION: [(col("homeScore")) > (100)]


In [25]:
# Execute lazy query
start = time.perf_counter()
result = query.collect()
polars_lazy_time = time.perf_counter() - start
print(f"\nPolars Lazy execution: {polars_lazy_time:.6f}s")
result


Polars Lazy execution: 0.016740s


hometeamName,avg_score,games
str,f64,u32
"""Nationals""",119.097792,317
"""Royals""",117.903553,591
"""Nuggets""",117.476751,1613
"""Warriors""",116.091747,2387
"""Celtics""",116.052871,2421
"""Pelicans""",115.813842,419
"""Lakers""",115.770459,2505
"""Suns""",115.168999,1929
"""76ers""",114.974261,1826


---

## 9. Summary: Key Differences

In [26]:
summary_data = {
    "Feature": ["Loading CSV", "Select Columns", "Filter Rows", "Group By + Agg", "Add Columns", "Handle Nulls", "Lazy Evaluation"],
    "Polars Syntax": [
        'pl.read_csv("file.csv")',
        'df.select(["col1", "col2"])',
        'df.filter(pl.col("col") > value)',
        'df.group_by("col").agg(...)',
        'df.with_columns(new_col)',
        'df.fill_null() / df.drop_nulls()',
        'pl.scan_csv().filter().collect()'
    ],
    "Pandas Syntax": [
        'pd.read_csv("file.csv")',
        'df[["col1", "col2"]]',
        'df[df["col"] > value]',
        'df.groupby("col").agg(...)',
        'df["new_col"] = ...',
        'df.fillna() / df.dropna()',
        'N/A (eager only)'
    ]
}
summary_df = pl.DataFrame(summary_data)
summary_df

Feature,Polars Syntax,Pandas Syntax
str,str,str
"""Loading CSV""","""pl.read_csv(""file.csv"")""","""pd.read_csv(""file.csv"")"""
"""Select Columns""","""df.select([""col1"", ""col2""])""","""df[[""col1"", ""col2""]]"""
"""Filter Rows""","""df.filter(pl.col(""col"") > valu…","""df[df[""col""] > value]"""
"""Group By + Agg""","""df.group_by(""col"").agg(...)""","""df.groupby(""col"").agg(...)"""
"""Add Columns""","""df.with_columns(new_col)""","""df[""new_col""] = ..."""
"""Handle Nulls""","""df.fill_null() / df.drop_nulls…","""df.fillna() / df.dropna()"""
"""Lazy Evaluation""","""pl.scan_csv().filter().collect…","""N/A (eager only)"""


In [27]:
print("\n✅ Polars advantages:")
print("- Faster execution (often 2-10x)")
print("- Lazy evaluation for optimized queries")
print("- More consistent API")
print("- Better memory efficiency")
print("- Native Rust backend")

print("\n⚠️ Pandas advantages:")
print("- More mature ecosystem")
print("- More tutorials/documentation")
print("- Better integration with some ML libraries")


✅ Polars advantages:
- Faster execution (often 2-10x)
- Lazy evaluation for optimized queries
- More consistent API
- Better memory efficiency
- Native Rust backend

⚠️ Pandas advantages:
- More mature ecosystem
- More tutorials/documentation
- Better integration with some ML libraries
